In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
data = np.load("/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/EEG_Analysis/P10/obj_presentationP10_02.npy")
labels = np.load("/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/EEG_Analysis/P10/triallabel.npy")
data = np.transpose(data, (0, 2, 1))

print(data.shape)
print(labels.shape)

(77, 64, 4096)
(77,)


In [86]:
# """Downsample the Data - as low pass fiter is already applied at 100Hz which is below the Nyquist freq., we dont have to worry about aliasing the data for the target freq. of 265 Hz """

# from scipy.signal import resample

# # Resample the data from 512 Hz to 256 Hz
# n_samples = data.shape[-1] // 2  # Divide the number of samples by 2 to downsample
# downsampled_data = resample(data, n_samples, axis=-1)
# print(downsampled_data.shape)
# # data = downsampled_data

## Feature Engineering

-  Time-Domain Feature Extraction:
This function extracts common statistical features from the time domain of the EEG signal for each epoch:

    - Mean: Average of the signal.
    - Standard Deviation: Spread of the signal values
    - Skewness: Asymmetry in the signal’s distribution
    - Kurtosis: "Tailedness" or sharpness of the peak of the signal distribution.

- Frequency-Domain Feature Extraction:
    - Power in Frequency Bands: Calculates theta (4-8 Hz), alpha (8-13 Hz), and beta (13-30 Hz) power from the Power Spectral Density (PSD).
    - Mean Frequency: Weighted average of frequencies.
    - Peak Frequency: Frequency where the maximum power occurs.

- Alpha Asymmetry:
    - Alpha power (8-13 Hz) is computed for both the left and right channels using Welch's method.
    - The logarithmic difference between alpha power in the two channels is calculated to measure asymmetry.


In [4]:
from scipy import stats
from scipy.signal import welch
from scipy import signal
import numpy as np
import pandas as pd

# Preprocessing
# Remove bad channels

    

# Time-Domain Features

def extract_time_domain_features(epoch):
    mean = np.mean(epoch, axis=-1)
    std = np.std(epoch, axis=-1)
    skewness = stats.skew(epoch, axis=-1)
    kurtosis = stats.kurtosis(epoch, axis=-1)
    
    features = {
        'Mean': [mean],
        'StdDev': [std],
        'Skewness': [skewness],
        'Kurtosis': [kurtosis],
    }
    
    return pd.DataFrame(features)

# Frequency-Domain Features
def extract_frequency_domain_features(epoch, sample_rate):
    freqs, psd = signal.welch(epoch, fs=sample_rate, axis=-1)
    
    # delta_power = np.sum(psd[(freqs >= 0.5) & (freqs <= 4)], axis=-1)
    theta_power = np.sum(psd[(freqs >= 4) & (freqs <= 8)], axis=-1)
    alpha_power = np.sum(psd[(freqs >= 8) & (freqs <= 13)], axis=-1)
    beta_power = np.sum(psd[(freqs >= 13) & (freqs <= 30)], axis=-1)
    # gamma_power = np.sum(psd[(freqs >= 30)], axis=-1)
    
    mean_freq = np.sum(freqs * psd, axis=-1) / np.sum(psd, axis=-1)
    peak_freq = freqs[np.argmax(psd, axis=-1)]
    
    features = {
        # 'DeltaPower': [delta_power],
        'ThetaPower': [theta_power],
        'AlphaPower': [alpha_power],
        'BetaPower': [beta_power],
        # 'GammaPower': [gamma_power],
        'MeanFrequency': [mean_freq],
        'PeakFrequency': [peak_freq]
    }
    
    return pd.DataFrame(features)

def extract_alpha_asymmetry(epoch, sample_rate, left_channel, right_channel):
    freqs, psd_left = signal.welch(epoch[left_channel], fs=sample_rate)
    _, psd_right = signal.welch(epoch[right_channel], fs=sample_rate)
    
    alpha_power_left = np.sum(psd_left[(freqs >= 8) & (freqs <= 13)])
    alpha_power_right = np.sum(psd_right[(freqs >= 8) & (freqs <= 13)])
    
    alpha_asymmetry = np.log(alpha_power_left + 1e-10) - np.log(alpha_power_right + 1e-10)
    
    return alpha_asymmetry

In [5]:
import re
    

def extract_features_from_eeg(eeg_data, 
                              sample_rate, 
                              ch_names, 
                              bad_channels,
                              tmax = None,
                              ch_inclusion_re = '.*',
                              left_right_pairs = None
                              ):
    
    # time-domain cutoff
    if tmax:
        eeg_data = eeg_data[:, :, :int(tmax*sample_rate)]

    printed = False
    n_trials, n_channels, n_times = eeg_data.shape
    features = []
    channes_used = set([])
    
    for i in range(n_trials):
        trial_features = []

        # Time and Frequency domain features
        for j in range(n_channels):
            # skip bad channels
            if ch_names[j] in bad_channels or not re.match(ch_inclusion_re, ch_names[j]):
                continue
            else:
                channes_used.add(ch_names[j])

            epoch = eeg_data[i, j, :]

            time_features = extract_time_domain_features(epoch)
            freq_features = extract_frequency_domain_features(epoch, sample_rate)
            
            channel_features = pd.concat([time_features, freq_features], axis=1)
            channel_features = channel_features.add_suffix(f'_{ch_names[j]}')
            trial_features.append(channel_features)
        
        if not printed:
            print('features per channel:', trial_features[0].shape)
            print('num channels used:', len(trial_features))
            printed = True
            
        # Alpha Asymmetry feature
        if left_right_pairs:
            for left_channel_name, right_channel_name in left_right_pairs:
                if left_channel_name in ch_names and right_channel_name in ch_names:
                    left_idx = ch_names.index(left_channel_name)
                    right_idx = ch_names.index(right_channel_name)
                    alpha_asymmetry = extract_alpha_asymmetry(eeg_data[i], sample_rate, left_idx, right_idx)
                    asymmetry_feature = pd.DataFrame({'AlphaAsymmetry': [alpha_asymmetry]})
                    asymmetry_feature = asymmetry_feature.add_suffix(f'_{left_channel_name}-{right_channel_name}')
                    trial_features.append(asymmetry_feature)
    
        features.append(pd.concat(trial_features, axis=1))

    features = pd.concat(features, axis=0)
    
    return features, channes_used

In [6]:
data.shape

(77, 64, 4096)

In [7]:
# Example parameters
import numpy as np
from scipy import stats


# data = np.load(data_path + 'P03_01.npy')
# labels_path = '/Users/arnavkapur/Desktop/Analysis_Imagery/imagery2024/PREPROCESSED_DATA/S01/class_labels/'
# labels = np.load(labels_path + 'group_P03_01.npy')

"""Choose for out grouped by labels"""
# data = np.load("/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/PREPROCESSED_DATA/2D/eeg/P03_2D.npy")
# labels = np.load('/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/PREPROCESSED_DATA/2D/class_labels/group_P03_2D.npy')
# # data = np.transpose(data, (0, 2, 1))

"""Choose for out grouped by trials """
data = np.load("/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/PREPROCESSED_DATA/2D/Saved_byTrial/eeg/eeg_P03_2D.npy")
labels = np.load('/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/PREPROCESSED_DATA/2D/Saved_byTrial/label/group_P03_2D.npy')

sample_rate = 512  # Hz
ch_names = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 
                    'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 
                    'P4', 'P8', 'POz', 'O1', 'O2', 'EOG', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 
                    'F6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CP4', 'P5', 'P1', 'P2', 
                    'P6', 'PO5', 'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Oz']
bad_channels = ['EOG']  # No bad channels in this example
tmax = 8  # Use the first 1 seconds of data (optional)
ch_inclusion_re = '.*'  #'O1|O2|Oz|POz|PO3|PO4|Pz|P3|P4|PO7' # # Include all channels  'O1|O2|Oz'
left_right_pairs = [('O1'), ('O2')]  # Example left-right pairs for alpha asymmetry


# Extract features
final_features, channels_used = extract_features_from_eeg(
    eeg_data=data,
    sample_rate=sample_rate,
    ch_names=ch_names,
    bad_channels=bad_channels,
    tmax=tmax,
    ch_inclusion_re=ch_inclusion_re,
    left_right_pairs=left_right_pairs
)

# final_features will be a DataFrame containing all the extracted features
print(final_features.head())
print('Channels used:', channels_used)


features per channel: (1, 9)
num channels used: 63
   Mean_Fp1  StdDev_Fp1  Skewness_Fp1  Kurtosis_Fp1  ThetaPower_Fp1  \
0  0.235964    1.202949      1.463989      0.955751        0.177205   
0  0.075508    0.799724      2.155030      4.038741        0.064566   
0  0.100267    0.922660      2.275815      4.313613        0.128895   
0  0.061135    0.542723      3.270230     10.743816        0.031897   
0 -0.018494    0.715766      3.211309     10.218731        0.093647   

   AlphaPower_Fp1  BetaPower_Fp1  MeanFrequency_Fp1  PeakFrequency_Fp1  \
0        0.014412       0.001587           3.324627                2.0   
0        0.006961       0.001094           3.312412                2.0   
0        0.012412       0.002309           3.156583                2.0   
0        0.001384       0.000916           3.436963                2.0   
0        0.007780       0.001177           3.161753                2.0   

   Mean_Fpz  ...  PeakFrequency_PO8   Mean_Oz  StdDev_Oz  Skewness_Oz  \
0  0

In [9]:
from scipy.signal import resample
import numpy as np
from scipy.signal import welch

#Get Data
# data_path = '/Users/arnavkapur/Desktop/Analysis_Imagery/imagery2024/PREPROCESSED_DATA/S01/eeg/'

# data = np.load(data_path + 'P03_01.npy')


data = np.load('/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/PREPROCESSED_DATA/Sven_Prp/eeg/eeg_P03_2D.npy')
labels = np.load('/Users/arnavkapur/Desktop/Analysis_3DImagery/imagery2024/PREPROCESSED_DATA/Sven_Prp/group_label/group_P03_2D.npy')


""" Use the first 500ms of the data and downsample it for classfiacation"""

# # Extract the first 256 samples (which corresponds to the first 500 ms)
# first_500ms_data = data[:, :, :256]
# print("New data shape:", first_500ms_data.shape)  # This should print (468, 64, 256)


# # # # Resample the data from 512 Hz to 256 Hz
# n_samples = data.shape[-1] // 2  # Divide the number of samples by 2 to downsample
# downsampled_data = resample(data, n_samples, axis=-1)
# print(downsampled_data.shape)
# data = downsampled_data

fs = 512

# Define frequency bands
delta_band = (1, 4)
theta_band = (4, 8)
alpha_band = (8, 12)
beta_band = (12, 30)
gamma_band = (30, 100)
sigma_band = (12, 16)




# Function to calculate band power
def bandpower(data, sf, band):
    freqs, psd = welch(data, sf, nperseg=fs)
    idx_band = np.logical_and(freqs >= band[0], freqs <= band[1])
    return np.mean(psd[idx_band])

# Calculate mean and peak frequency
def mean_and_peak_frequency(data, sf):
    freqs, psd = welch(data, sf, nperseg=fs)
    mean_freq = np.sum(freqs * psd) / np.sum(psd)
    peak_freq = freqs[np.argmax(psd)]
    return mean_freq, peak_freq

# Initialize feature arrays
mean_features = np.zeros((data.shape[0], data.shape[1]))
std_features = np.zeros((data.shape[0], data.shape[1]))
delta_band_features = np.zeros((data.shape[0], data.shape[1]))
theta_band_features = np.zeros((data.shape[0], data.shape[1]))
alpha_band_features = np.zeros((data.shape[0], data.shape[1]))
beta_band_features = np.zeros((data.shape[0], data.shape[1]))
gamma_band_features = np.zeros((data.shape[0], data.shape[1]))
sigma_band_features = np.zeros((data.shape[0], data.shape[1]))

mean_freq_features = np.zeros((data.shape[0], data.shape[1]))
peak_freq_features = np.zeros((data.shape[0], data.shape[1]))



## Alpha asymmetry
channel_pairs = [(0, 1)]  # Replace with actual pairs for your setup
alpha_asymmetry_features = np.zeros((data.shape[0], len(channel_pairs)))




# Set up wavelet scattering
# J max scale ogf wavelet transform
# # Q control the number of waveles per octave
# scattering = Scattering1D(J=6, shape=data.shape[2], Q=8)  # J is log2(N) with N=signal length =4096 -> Jmax = 12,
# example_input = data[0, 0, :]
# scattering_coeffs_example = scattering(example_input)
# scattering_features = np.zeros((data.shape[0], data.shape[1], scattering_coeffs_example.shape[0],scattering_coeffs_example.shape[1] ))


# Compute features per channel
for trial in range(data.shape[0]):
    for channel in range(data.shape[1]):
        channel_data = data[trial, channel, :]

        mean_features[trial, channel] = np.mean(channel_data)
        std_features[trial, channel] = np.std(channel_data)

        # Band intensities
        delta_band_features[trial, channel] = bandpower(channel_data, fs, delta_band)
        theta_band_features[trial, channel] = bandpower(channel_data, fs, theta_band)
        alpha_band_features[trial, channel] = bandpower(channel_data, fs, alpha_band)
        beta_band_features[trial, channel] = bandpower(channel_data, fs, beta_band)
        gamma_band_features[trial, channel] = bandpower(channel_data, fs, gamma_band)
        sigma_band_features[trial, channel] = bandpower(channel_data, fs, sigma_band)

         # Mean and peak frequency
        mean_freq_features[trial, channel], peak_freq_features[trial, channel] = mean_and_peak_frequency(channel_data, fs)


        # # Wavelet scattering
        # scattering_coeffs = scattering(channel_data)
        # scattering_features[trial, channel, :] = scattering_coeffs

         # Compute alpha asymmetry for defined channel pairs
    for idx, (ch1, ch2) in enumerate(channel_pairs):
        alpha_asymmetry_features[trial, idx] = alpha_band_features[trial, ch1] - alpha_band_features[trial, ch2]


combined_features = np.concatenate([
    mean_features,
    std_features,
    delta_band_features,
    theta_band_features,
    alpha_band_features,
    beta_band_features,
    gamma_band_features,
    sigma_band_features,
    mean_freq_features,
    peak_freq_features,
    alpha_asymmetry_features,
    # scattering_features.reshape(data.shape[0], -1)  # Flatten the scattering features
], axis=1)

print("Combined features shape:", combined_features.shape)

## alpha assymetry
## Mean and peak frequency




/Users/arnavkapur/.pyenv/versions/3.10.12/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 64, using nperseg = 64
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/arnavkapur/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/arnavkapur/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/w2/bjrs7bh539701pd5yx65p_cc0000gp/T/ipykernel_81354/2863423474.py:50: RuntimeWarning: invalid value encountered in scalar divide
  mean_freq = np.sum(freqs * psd) / np.sum(psd)


In [8]:
import numpy as np
import pandas as pd

# Example array with NaN values


def interpolate_nan_rows(arr):
    df = pd.DataFrame(arr)
    # Interpolate NaN values row-wise
    df = df.interpolate(method='linear', axis=1, limit_direction='both')
    return df.to_numpy()

interpolated_features = interpolate_nan_rows(combined_features)


NameError: name 'combined_features' is not defined

## SVM

In [141]:
final_features.shape

(1170, 567)

In [165]:
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler




# labels_path = '/Users/arnavkapur/Desktop/Analysis_Imagery/imagery2024/PREPROCESSED_DATA/S01/class_labels/'
# labels = np.load(labels_path + 'group_P03_01.npy')
# # Load data
X_data = final_features
y_labels = labels

# Split into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_labels, test_size=0.2, random_state=33, stratify=y_labels)

# X_train, X_temp, y_train, y_temp = train_test_split(X_data, y_labels, test_size=0.3, random_state=42, stratify=y_labels)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(svm.SVC(probability=True, class_weight='balanced'), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

model = grid_search.best_estimator_

# Predict class labels and probabilities
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

loss = log_loss(y_test, y_pred_proba)
print(f"Loss: {loss:.6f}")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))


Best parameters: {'C': 0.01, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 23.08%
Loss: 1.775007
Confusion Matrix:
[[ 6  9  4 10  6  4]
 [ 7 16  2  6  7  1]
 [ 4  7  5  7 10  6]
 [ 9  3  7  6  8  6]
 [ 7  7  7  5  6  7]
 [ 2  7  4  6  5 15]]
Classification Report:
              precision    recall  f1-score   support

           1       0.17      0.15      0.16        39
           2       0.33      0.41      0.36        39
           3       0.17      0.13      0.15        39
           4       0.15      0.15      0.15        39
           5       0.14      0.15      0.15        39
           6       0.38      0.38      0.38        39

    accuracy                           0.23       234
   macro avg       0.22      0.23      0.23       234
weighted avg       0.22      0.23      0.23       234


In [ ]:
def extract_time_features(data, eeg_info, time_points):

    ch_names = eeg_info['ch_names']
    sfreq = eeg_info['sfreq']

    print(time_points)
    columns = []
    for ch in ch_names:
        for tp in time_points:
            columns.append(f'{ch}_{tp:.2f}')
    print(len(columns))
    features = {col: [] for col in columns}

    for trial in data:
        for i, eeg_channel in enumerate(trial):
            for tp in time_points:
                features[f'{ch_names[i]}_{tp:.2f}'].append(eeg_channel[int(tp*sfreq)])

    return pd.DataFrame(features)





In [ ]:
import numpy as np
from scipy import stats, signal


# Time-Domain Features
def extract_time_domain_features(epoch):
    mean = np.mean(epoch, axis=-1)
    std = np.std(epoch, axis=-1)
    skewness = stats.skew(epoch, axis=-1)
    kurtosis = stats.kurtosis(epoch, axis=-1)
    
    features = {
        'Mean': [mean],
        'StdDev': [std],
        'Skewness': [skewness],
        'Kurtosis': [kurtosis],
    }
    
    return pd.DataFrame(features)

# Frequency-Domain Features
def extract_frequency_domain_features(epoch, sample_rate):
    freqs, psd = signal.welch(epoch, fs=sample_rate, axis=-1)
    
    # delta_power = np.sum(psd[(freqs >= 0.5) & (freqs <= 4)], axis=-1)
    theta_power = np.sum(psd[(freqs >= 4) & (freqs <= 8)], axis=-1)
    alpha_power = np.sum(psd[(freqs >= 8) & (freqs <= 13)], axis=-1)
    beta_power = np.sum(psd[(freqs >= 13) & (freqs <= 30)], axis=-1)
    # gamma_power = np.sum(psd[(freqs >= 30)], axis=-1)
    
    mean_freq = np.sum(freqs * psd, axis=-1) / np.sum(psd, axis=-1)
    peak_freq = freqs[np.argmax(psd, axis=-1)]
    
    features = {
        # 'DeltaPower': [delta_power],
        'ThetaPower': [theta_power],
        'AlphaPower': [alpha_power],
        'BetaPower': [beta_power],
        # 'GammaPower': [gamma_power],
        'MeanFrequency': [mean_freq],
        'PeakFrequency': [peak_freq]
    }
    
    return pd.DataFrame(features)

def extract_alpha_asymmetry(epoch, sample_rate, left_channel, right_channel):
    freqs, psd_left = signal.welch(epoch[left_channel], fs=sample_rate)
    _, psd_right = signal.welch(epoch[right_channel], fs=sample_rate)
    
    alpha_power_left = np.sum(psd_left[(freqs >= 8) & (freqs <= 13)])
    alpha_power_right = np.sum(psd_right[(freqs >= 8) & (freqs <= 13)])
    
    alpha_asymmetry = np.log(alpha_power_left + 1e-10) - np.log(alpha_power_right + 1e-10)
    
    return alpha_asymmetry

In [ ]:
import re

def extract_features_from_eeg(eeg_data, 
                              sample_rate, 
                              ch_names, 
                              bad_channels,
                              tmax = None,
                              ch_inclusion_re = '.*',
                              left_right_pairs = None
                              ):
    
    # time-domain cutoff
    if tmax:
        eeg_data = eeg_data[:, :, :int(tmax*sample_rate)]

    printed = False
    n_trials, n_channels, n_times = eeg_data.shape
    features = []
    channes_used = set([])
    
    for i in range(n_trials):
        trial_features = []

        # Time and Frequency domain features
        for j in range(n_channels):
            # skip bad channels
            if ch_names[j] in bad_channels or not re.match(ch_inclusion_re, ch_names[j]):
                continue
            else:
                channes_used.add(ch_names[j])

            epoch = eeg_data[i, j, :]

            time_features = extract_time_domain_features(epoch)
            freq_features = extract_frequency_domain_features(epoch, sample_rate)
            
            channel_features = pd.concat([time_features, freq_features], axis=1)
            channel_features = channel_features.add_suffix(f'_{ch_names[j]}')
            trial_features.append(channel_features)
        
        if not printed:
            print('features per channel:', trial_features[0].shape)
            print('num channels used:', len(trial_features))
            printed = True
            
        # Alpha Asymmetry feature
        if left_right_pairs:
            for left_channel_name, right_channel_name in left_right_pairs:
                if left_channel_name in ch_names and right_channel_name in ch_names:
                    left_idx = ch_names.index(left_channel_name)
                    right_idx = ch_names.index(right_channel_name)
                    alpha_asymmetry = extract_alpha_asymmetry(eeg_data[i], sample_rate, left_idx, right_idx)
                    asymmetry_feature = pd.DataFrame({'AlphaAsymmetry': [alpha_asymmetry]})
                    asymmetry_feature = asymmetry_feature.add_suffix(f'_{left_channel_name}-{right_channel_name}')
                    trial_features.append(asymmetry_feature)
    
        features.append(pd.concat(trial_features, axis=1))

    features = pd.concat(features, axis=0)
    
    return features, channes_used


In [ ]:
left_right_pairs = [
    ('F3', 'F4'),  
    ('F7', 'F8'),  
    ('Fp1', 'Fp2'), 
    ('C3', 'C4'),  
    ('P3', 'P4'),  
    ('O1', 'O2'), 
    ('T7', 'T8'),  
    ('P5', 'P6')   
]

In [ ]:
# used unstandardized data for this

sfreq = 512

features, channels_used = extract_features_from_eeg(eeg_epochs, sfreq, ch_names, bad_channels,
                                     tmax=10,
                                    #  ch_inclusion_re='.*[FT].*',
                                     left_right_pairs=left_right_pairs
                                    )
print("Extracted Features Shape:", features.shape)  # (n_trials, n_features)

In [ ]:
features

## Logistic Regression

In [19]:
from sklearn.linear_model import LogisticRegression

# Hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': [500,1000],
    'penalty': ['l2']}

grid_search = GridSearchCV(LogisticRegression(multi_class='ovr', class_weight='balanced'), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

model = grid_search.best_estimator_

# Predict class labels and probabilities
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

loss = log_loss(y_test, y_pred_proba)
print(f"Loss: {loss:.6f}")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
print(f"ROC-AUC Score: {roc_auc:.2f}")

# # ROC Curve
# fpr, tpr, _ = roc_curve(y_test, y_pred_proba[:, 1], pos_label=1)
# plt.plot(fpr, tpr, marker='.')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.show()

/Users/arnavkapur/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/arnavkapur/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/arnavkapur/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/arnavkapur/.pyenv/

Best parameters: {'C': 1, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy: 29.79%
Loss: 3.551564
Confusion Matrix:
[[5 5 2 4 0 0]
 [4 5 2 0 2 2]
 [2 3 1 3 4 3]
 [1 0 4 7 1 3]
 [2 3 4 1 4 2]
 [1 3 3 1 1 6]]
Classification Report:
              precision    recall  f1-score   support

           1       0.33      0.31      0.32        16
           2       0.26      0.33      0.29        15
           3       0.06      0.06      0.06        16
           4       0.44      0.44      0.44        16
           5       0.33      0.25      0.29        16
           6       0.38      0.40      0.39        15

    accuracy                           0.30        94
   macro avg       0.30      0.30      0.30        94
weighted avg       0.30      0.30      0.30        94

ROC-AUC Score: 0.67


## Random Forest Classificaiton

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Load data
X_data = combined_features
y_labels = labels

# Split into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_labels, test_size=0.2, random_state=33, stratify=y_labels)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=33), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

model = grid_search.best_estimator_

# Predict class labels and probabilities
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

loss = log_loss(y_test, y_pred_proba)
print(f"Loss: {loss:.6f}")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))


Best parameters: {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Accuracy: 30.85%
Loss: 1.665736
Confusion Matrix:
[[8 1 0 1 6 0]
 [3 3 4 1 3 1]
 [1 1 1 0 7 6]
 [3 0 2 5 2 4]
 [5 3 0 1 4 3]
 [2 0 1 1 3 8]]
Classification Report:
              precision    recall  f1-score   support

           1       0.36      0.50      0.42        16
           2       0.38      0.20      0.26        15
           3       0.12      0.06      0.08        16
           4       0.56      0.31      0.40        16
           5       0.16      0.25      0.20        16
           6       0.36      0.53      0.43        15

    accuracy                           0.31        94
   macro avg       0.32      0.31      0.30        94
weighted avg       0.32      0.31      0.30        94


## Try Classificaiton for 2 classes


In [ ]:
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Load your features and labels
X_data = final_features
y_labels = labels

# Filter to include only two classes (e.g., classes 0 and 1)
selected_classes = [1,2]  # Replace these with the actual class labels you want to classify
filter_mask = np.isin(y_labels, selected_classes)

X_data_filtered = X_data[filter_mask]
y_labels_filtered = y_labels[filter_mask]

# Optionally, you can relabel the classes to be 0 and 1 if they aren't already
# y_labels_filtered = np.where(y_labels_filtered == selected_classes[0], 0, 1)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data_filtered, y_labels_filtered, test_size=0.2, random_state=33, stratify=y_labels_filtered)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(svm.SVC(probability=True, class_weight='balanced'), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

model = grid_search.best_estimator_

# Predict class labels and probabilities
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

loss = log_loss(y_test, y_pred_proba)
print(f"Loss: {loss:.6f}")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))
